En este script vamos a obtener las imágenes segmentadas de 96*96 metros (384,384) que coindicen gráficamente y con el mismo nombre de fichero que las imágenes jpg.

El primer paso es leer los shapefiles y crear un fichero geotiff con las mismas geométricas con valor 255 y mismo sistema de coordenadas.

Una vez tengamos estos fichero de geotiff , lo vamos a trocear en un array de 10*10 imágenes tiff que coinciden con las imágenes jpg extraídas del WMS del PNOA tanto en contenido gráfico, como en  resolución(384,384) y en nombre de fichero.

Con estos dos grupos de ficheros obtendremos el dataset.


## Transformamos de Shape a Tiff

In [1]:
import rasterio
import shapefile
import numpy as np
import geopandas as gpd
import os.path
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from os import listdir

In [2]:
#leemos los ficheros del directorio /shape
archivos = listdir("./shape")
for archivo in archivos:
    nombre, extension = os.path.splitext(archivo)
    #Leemos solo ficheros extension .shp
    if (extension =='.shp'):
        #Guardamos los datos del shapefile en variable sf
        sf = gpd.read_file('./shape/' + archivo)
        
        #tamaño del raster en pixeles, 96 por matriz 10*10 = (9600,9600)
        shape = 9600, 9600
        
        #obtiene una transformacion Affine del .shp segun sus coordendas
        transform = rasterio.transform.from_bounds(*sf['geometry'].total_bounds, *shape)
        
        #rasteriza es una matriz de imágenes con geometrías de entrada sf
        rasterizado = rasterize(
            #Para cada geometria de sf se le assigna valor 255
            [(shape, 255) for shape in sf['geometry']],
            #out_shape es el array de salida
            out_shape = shape,
            #tranformaacion Affine declarada antes
            transform = transform,
            #Valor 0 para las areass sin geometria
            fill = 0,
            #Se tiene en cuenta todos los pixeles que toca la geometria, no solo los interiores
            all_touched = True,
            #el array de salida es uint8
            dtype = rasterio.uint8)
        
        #ficheros de salida con mismo nonmbre que el shapefile pero extensiuon tif
        filename = './imagenes/raster/' + nombre + '.tif'
        
        #guardamos los tif
        with rasterio.open(
            #Nombre del fcihero y tipo escritura
            filename, 'w',
            #Usamos diver para GeoTiff
            driver = 'GTiff',
            #el array es tipo uint8
            dtype = rasterio.uint8,
            #una unica banda
            count = 1,
            #Numero de filas
            width = shape[0],
            #Numero de columnas
            height = shape[1],
            #Transformacion Affine
            transform = transform
        ) as dst:
            dst.write(rasterizado, indexes = 1)
        print(filename, " creado")
        
#Fin proceso
print("Proceso finalizado ")

./imagenes/raster/cons_10_960.tif  creado
./imagenes/raster/cons_11_960.tif  creado
./imagenes/raster/cons_12_960.tif  creado
./imagenes/raster/cons_13_960.tif  creado
./imagenes/raster/cons_14_960.tif  creado
./imagenes/raster/cons_15_960.tif  creado
./imagenes/raster/cons_16_960.tif  creado
./imagenes/raster/cons_17_960.tif  creado
./imagenes/raster/cons_18_960.tif  creado
./imagenes/raster/cons_19_960.tif  creado
./imagenes/raster/cons_1_960.tif  creado
./imagenes/raster/cons_20_960.tif  creado
./imagenes/raster/cons_21_960.tif  creado
./imagenes/raster/cons_22_960.tif  creado
./imagenes/raster/cons_23_960.tif  creado
./imagenes/raster/cons_24_960.tif  creado
./imagenes/raster/cons_25_960.tif  creado
./imagenes/raster/cons_26_960.tif  creado
./imagenes/raster/cons_27_960.tif  creado
./imagenes/raster/cons_28_960.tif  creado
./imagenes/raster/cons_29_960.tif  creado
./imagenes/raster/cons_2_960.tif  creado
./imagenes/raster/cons_30_960.tif  creado
./imagenes/raster/cons_31_960.tif  c

## Creamos tiff de 96*96 metros

In [3]:
#numero de divisiones
NUM_DIVISIONES = 10
#Tamaño division
DIV = 960

#leemos los ficheros del directorio /shape
for archivo in archivos:
    nombre, extension = os.path.splitext(archivo)
    #Leemos solo ficheros extension .shp
    if (extension == '.shp'):
       #Guardamos los datos del shapefile en variable sf
        sf = shapefile.Reader("./shape/" + archivo)
     
        #parth imagenes raster
        path='./imagenes/raster/'+ nombre + '.tif'

        #leemos el ficheros tiff
        with rasterio.open(path) as ds:
            #Tranformacion Affine
            transformacion = ds.transform
            #array del tif
            raster = ds.read()
            #crs es el sistema de referencia de coordendas del fichero
            crs = ds.crs
        
        #Reordenamos los ejes de la matriz 
        raster = np.moveaxis(raster, (0, 1, 2,),(2 ,0 ,1 ))
    
        #Asigamos las coordenadas de las dimensiones del shapefile
        coord_x_min = round(sf.bbox[0], 2)
        coord_x_max = round(sf.bbox[2], 2)
        coord_y_min = round(sf.bbox[1], 2)
        coord_y_max = round(sf.bbox[3], 2)
       
        
        #Recorremos la matriz
        for i in range(NUM_DIVISIONES):
            for j in range(NUM_DIVISIONES):
                
                #Extraemos del array cortando el array original
                crop_img=raster[i*DIV : (i+1)*DIV, j*DIV : (j+1)*DIV, :]
                
                #calcumos coordendas cda imagen
                xmin = coord_x_min + (j * 96)
                xmax = xmin + 96
                ymax = coord_y_max - (i*96)
                ymin = ymax - 96

                #fichero tiff de salida con las coordendas x,y minimas y xymaximas
                filename='./imagenes/segmentadas/' + str(xmin) + '_' + str(ymin) + '_' + str(xmax) + '_' + str(ymax) + '.tiff'
                
                #guardamos los tif 96x96
                with rasterio.open(
                    #Nombre del fcihero y tipo escritura
                    filename, 'w',
                    #Usamos diver para GeoTiff
                    driver = 'GTiff',
                    #el array es tipo uint8
                    dtype = rasterio.uint8,
                    #una unica banda
                    count = 1,
                    #Numero de filas
                    width = 384,
                    #Numero de columnas
                    height = 384,
                    #Tranformacion Affine
                    transform = transform
                ) as dst:
                    #Grabamos imagen cortada
                    dst.write(crop_img[:,:,0], 1)
#Fin proceso
print("Proceso finalizado")

Proceso finalizado
